In [1]:
import livef1
from livef1.utils.constants import interpolation_map
from livef1.utils.helper import to_datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

session = livef1.get_session(season=2024, meeting_identifier="Spa", session_identifier="Race")

2025-01-21 16:48:22,732 : livef1 : INFO :: Selected meeting/session is:
	Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
	Meeting Name : Belgian Grand Prix
	Meeting Circuit Shortname : Spa-Francorchamps
2025-01-21 16:48:22,734 : livef1 : INFO :: The meeting was received successfully.
2025-01-21 16:48:22,736 : livef1 : INFO :: Selected meeting/session is:
	Session Name : Race
2025-01-21 16:48:22,736 : livef1 : INFO :: The session was received successfully.


In [2]:
session.generate()

2025-01-21 16:48:23,362 : livef1 : INFO :: 'Position.z' has not been found in lake, loading it.
2025-01-21 16:48:23,363 : livef1 : INFO :: Getting requested data : 'Position.z'.
	Selected session : 2024 Belgian Grand Prix Race
	Topic : Position.z
2025-01-21 16:48:26,545 : livef1 : INFO :: Data is successfully received.
2025-01-21 16:48:28,973 : livef1 : INFO :: Data is successfully parsed.
2025-01-21 16:48:30,026 : livef1 : INFO :: 'CarData.z' has not been found in lake, loading it.
2025-01-21 16:48:30,026 : livef1 : INFO :: Getting requested data : 'CarData.z'.
	Selected session : 2024 Belgian Grand Prix Race
	Topic : CarData.z
2025-01-21 16:48:35,140 : livef1 : INFO :: Data is successfully received.
2025-01-21 16:48:38,670 : livef1 : INFO :: Data is successfully parsed.
2025-01-21 16:48:42,446 : livef1 : INFO :: 'SessionStatus' has not been found in lake, loading it.
2025-01-21 16:48:42,446 : livef1 : INFO :: Getting requested data : 'SessionStatus'.
	Selected session : 2024 Belgian 

AttributeError: 'DataFrame' object has no attribute 'value'

In [37]:
session.data_lake.bronze_lake.get("SessionStatus")

,SessionKey,timestamp,status
0,9574,00:00:05.156,Inactive
1,9574,00:55:14.764,Started
2,9574,02:15:12.042,Finished
3,9574,02:17:07.716,Finalised
4,9574,02:17:07.716,Ends


In [3]:
laps = session.laps

In [4]:
res = session.get_data(dataName="Position")
df_pos = pd.DataFrame(res.value)
df_pos["Utc"] = to_datetime(df_pos["Utc"])
df_pos["timestamp"] = pd.to_timedelta(df_pos["timestamp"])

2025-01-21 16:08:13,322 : livef1 : INFO :: 'Position.z' has been found in lake.


In [5]:
res = session.get_data(dataName="Car_Data")
df_car = pd.DataFrame(res.value)
df_car["Utc"] = to_datetime(df_car["Utc"])
df_car["timestamp"] = pd.to_timedelta(df_car["timestamp"])

2025-01-21 16:08:16,119 : livef1 : INFO :: 'CarData.z' has been found in lake.


In [6]:
# display(df_pos[df_pos['DriverNo'] == "44"].iloc[168:1000].head(10))
# display(df_car[df_car['DriverNo'] == "44"].head(10))

# df_44 = df_pos[df_pos['DriverNo'] == "44"].reset_index()
# df_44["time_diff"] = (df_44["Utc"] - df_44["Utc"].shift(1)).dt.total_seconds()
# df_44[df_44["time_diff"] < 0.1]
# df_44.loc[108:118]
df = df_car.set_index(["DriverNo","Utc"]).join(df_pos.set_index(["DriverNo","Utc"]), rsuffix="_pos", how="outer").reset_index().sort_values(["DriverNo", "Utc"])

In [7]:
df

,DriverNo,Utc,SessionKey,timestamp,rpm,speed,n_gear,throttle,brake,drs,SessionKey_pos,timestamp_pos,Status,X,Y,Z
0,1,2024-07-28 12:10:22.788,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 00:01:45.570000,OnTrack,0.0,0.0,0.0
1,1,2024-07-28 12:10:23.108,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 00:01:45.570000,OnTrack,0.0,0.0,0.0
2,1,2024-07-28 12:10:23.308,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 00:01:45.570000,OnTrack,0.0,0.0,0.0
3,1,2024-07-28 12:10:23.548,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 00:01:45.570000,OnTrack,0.0,0.0,0.0
4,1,2024-07-28 12:10:23.928,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 00:01:46.630000,OnTrack,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228995,81,2024-07-28 14:25:44.870,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 02:17:06.893000,OnTrack,1380.0,2186.0,4139.0
1228996,81,2024-07-28 14:25:44.910,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,104.0,104.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN
1228997,81,2024-07-28 14:25:45.230,9574.0,0 days 02:17:07.573000,0.0,0.0,0.0,104.0,104.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN
1228998,81,2024-07-28 14:25:45.330,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,9574.0,0 days 02:17:07.353000,OnTrack,1380.0,2186.0,4139.0


In [26]:
laps_driver[["lap_start_date","lap_number"]].head().set_index("lap_start_date")

,lap_number
lap_start_date,
2024-07-28 13:03:52.742,1
2024-07-28 13:05:45.882,2
2024-07-28 13:07:35.731,3
2024-07-28 13:09:24.731,4
2024-07-28 13:11:14.596,5


In [79]:
df_driver = df[(df["DriverNo"] == "44")].set_index("Utc")
laps = session.laps
laps_driver = laps[laps["DriverNo"] == "44"]

for col in df_driver.columns:
    if col in interpolation_map:
        df_driver[col] = df_driver[col].interpolate(method = interpolation_map[col], order = 2).values


laps_driver["lap_end_date"] = laps_driver["lap_start_date"] + laps_driver["lap_time"] - timedelta(milliseconds=1)
laps_driver = pd.concat([laps_driver[["lap_start_date", "lap_number"]].set_index("lap_start_date"), laps_driver[["lap_end_date", "lap_number"]].set_index("lap_end_date")]).reset_index().sort_values("index").dropna()

        
df_driver = df_driver.join(laps_driver.set_index("index"), how="outer")
df_driver["lap_number"] = df_driver["lap_number"].ffill()
df_driver.index.names = ['Utc']



df_driver = df_driver.reset_index()
df_driver = df_driver[df_driver.Utc.between(laps_driver["index"].min(), laps_driver["index"].max())]

df_driver["SessionKey"] = df_driver["SessionKey"].ffill().bfill()
df_driver["timestamp"] = df_driver["Utc"] - session.first_datetime

C:\Users\EU10698\AppData\Local\Temp\ipykernel_12376\2618084168.py:7: FutureWarning:

Series.interpolate with method=ffill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\EU10698\AppData\Local\Temp\ipykernel_12376\2618084168.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [81]:
from plotly import express as px
df_temp = df_driver[df_driver["lap_number"] == 1]
df_temp["counter"] = np.arange(len(df_temp))

px.scatter(x = df_temp.X, y = df_temp.Y, color=df_temp.counter)

C:\Users\EU10698\AppData\Local\Temp\ipykernel_12376\757474580.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_temp = df_44.iloc[22000:23900][~df_44["SessionKey"].isnull()]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [ ]:
df_temp = df_44.iloc[22000:23900]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [ ]:
df_44["speed"] = df_44.reset_index()["speed"].astype(float).interpolate(method = "polynomial", order = 2).values
df_temp = df_44.iloc[22000:23500]
df_temp["fill"] = df_temp["DriverNo_pos"].isnull()
px.scatter(df_temp, x = df_temp.index, y = "speed", color="fill")

In [ ]:
res = session.get_data(dataName="Weather_Data")
df_weather = pd.DataFrame(res.value)
df_weather["timestamp"] = pd.to_datetime(df_weather["timestamp"])

In [ ]:
idx_cols = ["hour","min","sec","DriverNo"]
df = df_car.set_index(idx_cols).join(df_pos.set_index(idx_cols), rsuffix="_")

In [ ]:
c = pd.to_datetime(df_car[(df_car["min"] == 2) & (df_car["hour"] == 0)].timestamp).unique()
p = pd.to_datetime(df_w[(df_w["min"] == 2) & (df_w["hour"] == 0) & (df_w["sec"] > 27)].timestamp).unique()

for i in c-p:
    print(i.total_seconds())

In [ ]:
df_all = df.reset_index().set_index("timestamp").join(df_weather.set_index("timestamp"), rsuffix="_w_", how="outer")

In [ ]:
df_car[df_car["DriverNo"] == "44"]

In [ ]:
df.reset_index()[df.reset_index()["DriverNo"] == "44"]

In [ ]:
df_all[df_all["DriverNo"] == "44"].reset_index().iloc[44000:50000]

## Weather Data

In [ ]:
for col in interpolation_map:
    df_all[col] = df_all.reset_index()[col].astype(float).interpolate(method = interpolation_map[col], order = 2).values

In [ ]:
import matplotlib.pyplot as plt

df_temp = df_all[~df_all["DriverNo"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.index), df_temp["AirTemp"])
df_temp = df_all[df_all["DriverNo"].isnull()]
plt.scatter(pd.to_datetime(df_temp.index), df_temp["AirTemp"])
plt.show()

In [ ]:
df_all.dropna(subset="DriverNo")

## Position Data

In [ ]:
df_all.dtypes

In [ ]:
df_temp = df_all[~df_all["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.index), df_temp["speed"])
df_temp = df_all[df_all["X"].isnull()]
plt.scatter(pd.to_datetime(df_temp.index), df_temp["speed"])
plt.show()

In [ ]:
df_44 = df_all[df_all["DriverNo"] == "44"].reset_index().iloc[44000:50000]

df_temp = df_44[~df_44["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["speed"])

df_temp = df_44[df_44["X"].isnull()]
plt.scatter(df_temp.index, df_temp["speed"])
plt.show()


df_temp = df_44[~df_44["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.timestamp), df_temp["speed"])
plt.show()

In [ ]:
pd.to_datetime(df_44.timestamp).dt.time - pd.to_datetime(df_44.timestamp).shift(1).dt.time

In [ ]:
df_44.head(50)

In [ ]:
(pd.to_datetime(df_44.timestamp) - pd.to_datetime(df_44.timestamp).shift(1)).dt.total_microseconds()

In [ ]:
df_44

In [ ]:
for col in df_weather.columns[2:]:
    plt.figure(figsize=(20,5))
    plt.scatter(pd.to_datetime(df_all.index).values, df_all[col].astype(float), label = col)
    plt.legend()
    plt.show()

In [ ]:
na_list = [np.nan] * 100
test_ts = pd.Series([-1] + na_list + [-1] + na_list + [-1] + na_list + [0] + na_list + [1]+ na_list + [1] + na_list + [1])

plt.figure(figsize=(20,5))
plt.scatter(test_ts.index, test_ts, label = "plain")
plt.plot(test_ts.index, test_ts.interpolate(method = "linear"), label = "linear")
plt.show()

for method in ['nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial']:
    plt.figure(figsize=(20,5))
    plt.scatter(test_ts.index, test_ts, label = "plain")
    for d in [2,3,5]:
        plt.plot(test_ts.index, test_ts.interpolate(method = method, order = d), label = method + "_" + str(d))
    plt.legend()
    plt.show()

for method in ['krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline']:
    plt.figure(figsize=(20,5))
    plt.scatter(test_ts.index, test_ts, label = "plain")
    for d in [2,3,5]:
        plt.plot(test_ts.index, test_ts.interpolate(method = method, order = d), label = method + "_" + str(d))
    plt.legend()
    plt.show()

# plt.figure(figsize=(20,5))
# for d in [2,3,5]:
#     plt.plot(test_ts.index, test_ts.interpolate(method = "spline", order = d), label = "spline_" + str(d))
# plt.legend()
# plt.show()



In [ ]:
df_temp = df_all.copy()
df_temp.index = pd.to_datetime(df_all.index).values

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "linear"), label = "linear")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "polynomial", order = 2), label = "polynomial")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "spline", order = 2), label = "spline")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float), label = "plain")
plt.show()



In [ ]:
len(df_temp)
df_viz

In [ ]:
for d in [2,3,5,7,9,15,25,35]:
    try:
        df_viz = df_temp["WindSpeed"].astype(float).interpolate(method = "linear", order = d, limit_direction="both").reset_index().iloc[500000:1000000]
        print(d)
        plt.figure(figsize=(20,5))
        plt.scatter(df_viz["index"], df_viz["WindSpeed"], label = str(d))
        plt.show()
    except:
        pass

In [ ]:
df_all.reset_index()[~df_all.reset_index()["AirTemp"].isnull()]

In [ ]:
df_all.reset_index().iloc[2373112-25:2373112+25]

In [ ]:
import numpy as np

cols = df_weather.columns[2:]
df_all.loc[:,cols] = df_all.loc[:,cols].ffill()


In [ ]:
df_all.dropna(subset="Channels").dtypes

In [ ]:
df_all

In [ ]:
res = session.get_data(dataName="Weather_Data")
df_w = pd.DataFrame(res.value)

In [ ]:
res = session.get_data(dataName="Track_Status")
df = pd.DataFrame(res.value)
# df["Utc"] = pd.to_datetime(df["Utc"])
# df["duration"] = df["Utc"] - df["Utc"].shift(1)
df

In [ ]:
res = session.get_data(dataName="Tyre_Stints")
df = pd.DataFrame(res.value)
df

In [ ]:
df

In [ ]:
res = session.get_data(dataName="Weather_Data")
df = pd.DataFrame(res.value)
df

In [ ]:
from livef1.adapters.livetimingf1_adapter import livetimingF1_getdata

data = livetimingF1_getdata("https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/PitLaneTimeCollection.jsonStream", stream=True)
data

for key, value in data.items():
    if "_deleted" in value["PitTimes"].keys():
        for deleted_driver in value["PitTimes"]["_deleted"]:
            record = {
                "session_key": 0,
                "timestamp": key,
                "_deleted": deleted_driver
            }
            print(record)
            
    else:
        for driver_no, info in value["PitTimes"].items():
            print(info)
            record = {
                "session_key": 0,
                "timestamp": key,
                **info
            }
            print(record)
    
    print("----------")

In [ ]:
res = session.get_data(dataName="Pit_Lane_Times")
df = pd.DataFrame(res.value)
df

In [ ]:
import fastf1

session = fastf1.get_session(2024, 'Belgian', 'Race')
session.load()

In [ ]:
help(session)

In [ ]:
session.weather_data

In [ ]:
session.laps

In [ ]:
session.laps[~session.laps["PitInTime"].isnull()].sort_values("Time")

In [ ]:
session.laps[~session.laps["PitOutTime"].isnull()].sort_values("Time")

In [ ]:
session.race_control_messages

In [ ]:
session.results

In [ ]:
session.car_data["1"]